In [3]:
x = [1,2,3]
print(x[-1])

3


In [1]:

from FCSRN import fcsrn
#from FCSRN_AugLoss import fcsrn

import tensorflow as tf
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import time
import re
#import difflib
import edit_distance
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
HEIGHT = 48; WIDTH = 160; CHANNELS = 3
IMAGE_SHAPE = (HEIGHT, WIDTH, CHANNELS)
EPOCHS = 100
BATCH_SIZE = 100
N_TRAIN = 4000
N_TEST = 1000

BATCHES_TRAIN = N_TRAIN / BATCH_SIZE
BATCHES_TEST = N_TEST / BATCH_SIZE

# load data + labels

In [3]:
# resize image and add padding on one dimensions to keep ratios
def resize_withPadding(image, targetWidth=WIDTH, targetHeight=HEIGHT):    
    inputWidth = image.shape[1]
    inputHeight = image.shape[0]
    # scale either width or height, depending on which scaling factor would be smaller
    scale_width = targetWidth / inputWidth
    scale_height = targetHeight / inputHeight
    
    if scale_width < scale_height:
        # scale width, pad height
        result = cv2.resize(image, dsize=(0,0), fx=scale_width, fy=scale_width)
        padding = targetHeight - result.shape[0]
        p_top = int(padding/2)
        p_bot = p_top if (padding%2) == 0 else p_top + 1
        assert padding >= 0 and (p_top + p_bot) == padding, "unexpected height-padding: %d"%(padding)
        result = cv2.copyMakeBorder(result, top=p_top, bottom=p_bot, left=0, right=0, borderType=cv2.BORDER_CONSTANT,value=0)
    else:
        # scale height, pad width
        result = cv2.resize(image, dsize=(0,0), fx=scale_height, fy=scale_height)
        padding = targetWidth - result.shape[1]
        p_left = int(padding/2)
        p_right = p_left if (padding%2) == 0 else p_left + 1
        assert padding >= 0 and (p_left + p_right) == padding, "unexpected width-padding: %d"%(padding)
        result = cv2.copyMakeBorder(result, top=0, bottom=0, left=p_left, right=p_right, borderType=cv2.BORDER_CONSTANT,value=0)                                    
    return result

# open and resize an image
def loadImage(imgPath, height=HEIGHT, width=WIDTH):
    image = cv2.imread(str(imgPath), cv2.IMREAD_COLOR)
    image = resize_withPadding(image, width, height)
    assert image.shape[0] == height and image.shape[1] == width, "resizing failed"
    return image

In [4]:
# load data
datasetPath = Path("C:/Users/andre/Desktop/m/datasets/SCUT-WMN DataSet")
trainfile = datasetPath / "difficult_samples_for_train.txt"
testfile = datasetPath / "difficult_samples_for_test.txt"

# load trainimages
df_train = pd.read_csv(trainfile, sep="[ ,]" ,header=None)
images_train = np.array([
    loadImage(datasetPath / row[0]) for row in df_train.values
]).astype("float32")
labels_train = np.array([
    row[1:] for row in df_train.values
]).astype("int")

# load testimages
df_test = pd.read_csv(testfile, sep="[ ,]" ,header=None)
images_test = np.array([
    loadImage(datasetPath / row[0]) for row in df_test.values
]).astype("float32")
labels_test = np.array([
    row[1:] for row in df_test.values
]).astype("int")

print(images_train.shape)
print(labels_train.shape)

C:\Users\andre\Miniconda3\envs\keras_gpu\lib\site-packages\ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys
C:\Users\andre\Miniconda3\envs\keras_gpu\lib\site-packages\ipykernel_launcher.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


(4000, 48, 160, 3)
(4000, 5)


In [5]:
# make tf-datasets
train_X = tf.data.Dataset.from_tensor_slices(images_train[0:N_TRAIN])
train_y = tf.data.Dataset.from_tensor_slices(labels_train[0:N_TRAIN])

test_X = tf.data.Dataset.from_tensor_slices(images_test[0:N_TEST])
test_y = tf.data.Dataset.from_tensor_slices(labels_test[0:N_TEST])

train_X = train_X \
    .cache() \
    .batch(BATCH_SIZE)
    
train_y = train_y \
    .cache() \
    .batch(BATCH_SIZE)

test_X = test_X \
    .cache() \
    .batch(BATCH_SIZE)
    
test_y = test_y \
    .cache() \
    .batch(BATCH_SIZE)


print(train_X.element_spec)
print(train_y.element_spec)

TensorSpec(shape=(None, 48, 160, 3), dtype=tf.float32, name=None)
TensorSpec(shape=(None, 5), dtype=tf.int32, name=None)


# build/load model

In [6]:
model_savepath = None#Path("D:/m2/savedmodels/FCSRN_with_initializer")
model = fcsrn(IMAGE_SHAPE, checkpoint_path=model_savepath)

No checkpointpath given, model will not be saved.


In [7]:
#model.model.summary()

# train 

In [8]:
#print("Training %d batches of size %d for %d epochs" % (BATCHES_TRAIN, BATCH_SIZE, EPOCHS) )
model.train(train_X, train_y, EPOCHS, BATCH_SIZE)

step 0 took: 6.723076 seconds
step 1 took: 0.722070 seconds
step 2 took: 0.945475 seconds
step 3 took: 0.945473 seconds
step 4 took: 0.984369 seconds
step 5 took: 0.965648 seconds
step 6 took: 0.949462 seconds
step 7 took: 0.959407 seconds
step 8 took: 0.959435 seconds
step 9 took: 0.953584 seconds
step 10 took: 0.965445 seconds
step 11 took: 0.954421 seconds
step 12 took: 0.938490 seconds
step 13 took: 0.938493 seconds
step 14 took: 0.933504 seconds
step 15 took: 0.938490 seconds
step 16 took: 0.956912 seconds


KeyboardInterrupt: 

# Evaluate 

In [9]:
total_characters = N_TEST * 5
lcr, ar, mismatches = model.accuracy(test_X, test_y, BATCH_SIZE, total_characters, N_TEST)
print("lcr: %f" % (lcr))
print("ar: %f" % (ar))

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
lcr: 0.856000
ar: 0.968600


In [10]:
####
# takes array of predicted-truth pairs where at least one predictionerror occured.
# outputs arrays containing which digits exactly where predicted wrong.
####
# mimatches: output from fcsrn.accuracy()
####
# returns: (
#    error_replace-array: (predicted digit, expected digit), predicted label, truthlabel
#    error_extra:         extra predicted digit, predicted label, truthlabel
#    error_missing:       missing predicted digit, predicted label, truthlabel
#)
####
def list_errors(mismatches):
    error_replace = [] # wrongly predicted digits
    error_extra = [] # errors where too many characters predicted
    error_missing = [] # too few characters predicted
    # iterate every sample with mispredictions
    for pred, truth, diff in mismatches:
        # ignore -1/BLANK digits at end of pred
        index_blank = np.argwhere(pred==-1)
        index_blank = index_blank[0,0] if len(index_blank) > 0 else None
        pred = pred[0:index_blank]
        
        # get minimal editdistance with ops
        matcher = edit_distance.SequenceMatcher(pred, truth)
        for op, i1,i2, j1,j2 in matcher.get_opcodes():
            if op == "replace":
                # one or more characters were not predicted correctly
                wrong_predictions = pred[i1:i2]
                expected_predictions = truth[j1:j2]
                for pair in zip(wrong_predictions,expected_predictions):
                    error_replace.append((pair, pred, truth))   
            elif op == "delete":
                # one or more characters more than necessary predicted
                extra_predictions = pred[i1:i2]            
                for extra_pred in extra_predictions:
                    assert extra_pred != -1, "blank encountered"                
                    error_extra.append((extra_pred, pred, truth))
            elif op == "insert":
                # one or more characters were not predicted at all
                missed_predictions = truth[j1:j2]
                for missing_pred in missed_predictions:
                    error_missing.append((missing_pred, pred, truth))
    error_replace = np.array(error_replace)
    error_extra = np.array(error_extra)
    error_missing = np.array(error_missing)
    return (error_replace, error_extra, error_missing)

In [11]:
err_replace, err_extra, err_missing = list_errors(mismatches)
print(err_replace.shape)
print(err_extra.shape)
print(err_missing.shape)

(148, 3)
(2, 3)
(9, 3)


In [53]:

mistakes = [
    row[0] for row in err_replace
]
# sort by truth
mistakes.sort(key=lambda row:row[1])
pp.pprint("predicted, truth")
pp.pprint(mistakes)

'predicted, truth'
[   (10, 0),
    (1, 0),
    (10, 0),
    (19, 0),
    (15, 0),
    (1, 0),
    (19, 0),
    (10, 0),
    (10, 0),
    (19, 0),
    (6, 0),
    (7, 1),
    (17, 1),
    (4, 1),
    (11, 2),
    (11, 2),
    (12, 2),
    (14, 2),
    (12, 2),
    (16, 2),
    (3, 2),
    (13, 3),
    (13, 3),
    (12, 3),
    (12, 3),
    (12, 3),
    (12, 3),
    (14, 4),
    (14, 4),
    (13, 4),
    (14, 4),
    (14, 4),
    (13, 4),
    (14, 4),
    (13, 4),
    (11, 4),
    (12, 5),
    (3, 5),
    (15, 5),
    (14, 5),
    (15, 5),
    (14, 5),
    (15, 6),
    (8, 6),
    (5, 6),
    (15, 6),
    (15, 6),
    (15, 6),
    (8, 6),
    (5, 6),
    (8, 6),
    (16, 6),
    (15, 6),
    (5, 6),
    (17, 7),
    (17, 7),
    (17, 7),
    (17, 7),
    (17, 7),
    (17, 7),
    (16, 7),
    (17, 7),
    (17, 7),
    (6, 8),
    (6, 8),
    (18, 8),
    (18, 8),
    (6, 8),
    (17, 8),
    (9, 8),
    (6, 8),
    (9, 8),
    (3, 8),
    (18, 8),
    (18, 8),
    (19, 9),
    (18, 9),


In [100]:
# number of occurences of truthvalues
truth_occurences = [pair[1] for pair in mistakes]
digit, count = np.unique(truth_occurences, return_counts=True)
percentages = np.round(100* count / sum(count),2)

count_truth = dict(zip(digit, zip(count, percentages)))
pp.pprint("truth: #, %")
pp.pprint(count_truth)
# order by count
print("truth:\t#,\t%\tDESC")
for key,value in sorted(count_truth.items(), key=lambda item: -item[1][0]):
    print("%d:\t%d\t%.2f%%"%(key,value[0],value[1]))

# relative occurence of all halve-digits
#sum_perc = 0
#for key,value in count_truth.items():
#    if key >= 10:
#        sum_perc += value[1]
#print("ErrPercentage of halfdigits: %2f"%(sum_perc))

'truth: #, %'
{   0: (11, 7.43),
    1: (3, 2.03),
    2: (7, 4.73),
    3: (6, 4.05),
    4: (9, 6.08),
    5: (6, 4.05),
    6: (12, 8.11),
    7: (9, 6.08),
    8: (12, 8.11),
    9: (10, 6.76),
    10: (4, 2.7),
    11: (9, 6.08),
    12: (9, 6.08),
    13: (13, 8.78),
    14: (2, 1.35),
    15: (2, 1.35),
    17: (5, 3.38),
    18: (12, 8.11),
    19: (7, 4.73)}
truth:	#,	%	DESC
13:	13	8.78%
6:	12	8.11%
8:	12	8.11%
18:	12	8.11%
0:	11	7.43%
9:	10	6.76%
4:	9	6.08%
7:	9	6.08%
11:	9	6.08%
12:	9	6.08%
2:	7	4.73%
19:	7	4.73%
3:	6	4.05%
5:	6	4.05%
17:	5	3.38%
10:	4	2.70%
1:	3	2.03%
14:	2	1.35%
15:	2	1.35%
ErrPercentage of halfdigits: 42.560000


In [102]:
# count digits in traindataset
digits, count = np.unique(labels_train.flatten(), return_counts=True)
percentages = np.round(100* count / sum(count),2)
digit_count = dict(zip(digits, zip(count, percentages)))

# order by count
print("digit:\t#,\t%\tDESC")
for key,value in sorted(digit_count.items(), key=lambda item: -item[1][0]):
    print("%d:\t%d\t%.2f%%"%(key,value[0],value[1]))


digit:	#,	%	DESC
0:	6612	33.06%
1:	3234	16.17%
8:	1538	7.69%
3:	1214	6.07%
2:	1100	5.50%
7:	1037	5.18%
9:	920	4.60%
5:	916	4.58%
6:	754	3.77%
4:	517	2.58%
13:	340	1.70%
18:	245	1.23%
17:	221	1.10%
19:	219	1.10%
12:	215	1.08%
11:	210	1.05%
16:	202	1.01%
15:	184	0.92%
14:	165	0.82%
10:	157	0.78%


In [112]:
# zip digits with number of errors and occurence in traindataset
digits = [d for d in range(0,20)]
err_perc = [
    count_truth.get(digit)[1] if count_truth.get(digit) != None else 0
    for digit in digits
]
perc_in_train = [
    digit_count.get(digit)[1]
    for digit in digits
]

print("digit\terror%\ttrain%")
for digit in digits:
    print("%d:\t%.2f\t%.2f"%(digit, err_perc[digit], perc_in_train[digit]))


digit	error%	train%
0:	7.43	33.06
1:	2.03	16.17
2:	4.73	5.50
3:	4.05	6.07
4:	6.08	2.58
5:	4.05	4.58
6:	8.11	3.77
7:	6.08	5.18
8:	8.11	7.69
9:	6.76	4.60
10:	2.70	0.78
11:	6.08	1.05
12:	6.08	1.08
13:	8.78	1.70
14:	1.35	0.82
15:	1.35	0.92
16:	0.00	1.01
17:	3.38	1.10
18:	8.11	1.23
19:	4.73	1.10


In [172]:
# trainpairs
pairCounts = {}
sum = 0
for pair in mistakes:
    if pairCounts.get(pair) ==  None:
        pairCounts[pair] = 1
        sum += 1
    else:
        pairCounts[pair] += 1
        sum += 1
# print all mistakepairs
print("errPair \terr#\terr%")    
for key,value in sorted(pairCounts.items(), key=lambda item: -item[1]):
    print(key, " \t%d\t%.2f%%"%(value,100*value/sum))
print("total: %d mistakes"%(sum))

# print ignoring misstate
print("ignoring misstate")
total = 0
for key,value in sorted(pairCounts.items(), key=lambda item: -item[1]):
    if not test_misstate(key[0], key[1]):
        print(key, " \t%d\t%.2f%%"%(value,100*value/sum))
        total += value
print("total: %d - %.2f%%"%(total, 100*total/sum))

# print only misstate
print("only misstate")
total = 0
for key,value in sorted(pairCounts.items(), key=lambda item: -item[1]):
    if test_misstate(key[0], key[1]):
        print(key, " \t%d\t%.2f%%"%(value,100*value/sum))
        total += value
print("total: %d - %.2f%%"%(total, 100*total/sum))

# print only those mistakes where state was mid
print("only lower state")
for key,value in sorted(pairCounts.items(), key=lambda item: -item[1]):
    if key[0] < 10 and key[1] < 10:
        print(key, " \t%d\t%.2f%%"%(value,100*value/sum))

errPair 	err#	err%
(4, 13)  	12	8.11%
(17, 7)  	8	5.41%
(3, 12)  	7	4.73%
(19, 9)  	6	4.05%
(14, 4)  	5	3.38%
(15, 6)  	5	3.38%
(10, 0)  	4	2.70%
(12, 3)  	4	2.70%
(6, 8)  	4	2.70%
(18, 8)  	4	2.70%
(2, 11)  	4	2.70%
(19, 0)  	3	2.03%
(13, 4)  	3	2.03%
(8, 6)  	3	2.03%
(5, 6)  	3	2.03%
(19, 18)  	3	2.03%
(9, 18)  	3	2.03%
(1, 0)  	2	1.35%
(11, 2)  	2	1.35%
(12, 2)  	2	1.35%
(13, 3)  	2	1.35%
(15, 5)  	2	1.35%
(14, 5)  	2	1.35%
(9, 8)  	2	1.35%
(18, 9)  	2	1.35%
(8, 9)  	2	1.35%
(1, 10)  	2	1.35%
(12, 11)  	2	1.35%
(17, 11)  	2	1.35%
(8, 17)  	2	1.35%
(0, 18)  	2	1.35%
(16, 18)  	2	1.35%
(0, 19)  	2	1.35%
(15, 19)  	2	1.35%
(15, 0)  	1	0.68%
(6, 0)  	1	0.68%
(7, 1)  	1	0.68%
(17, 1)  	1	0.68%
(4, 1)  	1	0.68%
(14, 2)  	1	0.68%
(16, 2)  	1	0.68%
(3, 2)  	1	0.68%
(11, 4)  	1	0.68%
(12, 5)  	1	0.68%
(3, 5)  	1	0.68%
(16, 6)  	1	0.68%
(16, 7)  	1	0.68%
(17, 8)  	1	0.68%
(3, 8)  	1	0.68%
(8, 10)  	1	0.68%
(0, 10)  	1	0.68%
(4, 11)  	1	0.68%
(2, 12)  	1	0.68%
(18, 12)  	1	0.68%
(3, 13)  	1	0.

In [171]:

def test_misstate(a,b):
    # if both same state, return false
    if (a<10 and b<10) or (a>=10 and b>=10):
        return False
    x = max(a,b);y = min(a,b)
    if x == 19 and y == 0:
        return True
    return (x - y) in [9,10]

for i in range(20):
    for j in range(20):
        if test_misstate(i,j):
            print("%d,%d: %s"%(i,j,test_misstate(i,j)))

0,10: True
0,19: True
1,10: True
1,11: True
2,11: True
2,12: True
3,12: True
3,13: True
4,13: True
4,14: True
5,14: True
5,15: True
6,15: True
6,16: True
7,16: True
7,17: True
8,17: True
8,18: True
9,18: True
9,19: True
10,0: True
10,1: True
11,1: True
11,2: True
12,2: True
12,3: True
13,3: True
13,4: True
14,4: True
14,5: True
15,5: True
15,6: True
16,6: True
16,7: True
17,7: True
17,8: True
18,8: True
18,9: True
19,0: True
19,9: True
